In [4]:
### Ejecutar en caso de que no esten instaladas las siguientes librerias
#
#BeautifulSoup: Para realizar el scraping de datos de la página web.
#Requests: Para hacer solicitudes HTTP a la página web.
#Selenium (opcional): Si la página web utiliza JavaScript para cargar contenido dinámico, puedes usar Selenium para interactuar con la página.
#MySQL Connector: Para interactuar con la base de datos MySQL.
import sys
!{sys.executable} -m pip install beautifulsoup4 requests selenium mysql-connector-python


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
# bloque para obtener el valor del dolar blue usando api
import requests

urlDolarBlue = "https://dolarapi.com/v1/dolares/blue"
headers = {"Accept": "application/json"}
try:
  resp =requests.get(urlDolarBlue)
  resp.raise_for_status()
  if resp.ok:
    data = resp.json()
    dolar_blue = int(data['venta'])
    print("El valor del dolar blue es: ", dolar_blue)
      
except requests.RequestException as error:
    print("Ocurrio un error: ", error)


El valor del dolar blue es:  800


In [44]:
# Bloque para obtener la fecha actual
from datetime import datetime

fecha_actual_cruda = datetime.now()

fecha_actual = fecha_actual_cruda.strftime('%Y-%m-%d')
print("La fecha actual es : ", fecha_actual)

La fecha actual es :  2023-10-02


In [45]:
# Bloque para realizar el scraping de libros
#import requests
from bs4 import BeautifulSoup
import re #Libreria para manejar expresiones regulares

# Lista para guardar los libros
top_100_libros = []
# Lista para los errores al acceder a los detalles del libro
error_top_100_libros = []
# Función para limpiar y formatear el precio
def format_price(price_str):
    price_str = re.sub(r'[^\d,]', '', price_str) #Expresión regular
    price_float = float(price_str.replace(',', '.')) #Me aseguro q sea float, sino tenia problemas al multiplicar por el precio dolar
    return round(price_float,2) # Redondeo a solo 2 decimales.

url = "https://cuspide.com/100-mas-vendidos/"


try:
    response = requests.get(url)
    response.raise_for_status()
    if response.ok:
        precio_dolar_blue = dolar_blue
        fecha = fecha_actual

        soup = BeautifulSoup(response.text, 'html.parser')
        # Encuentra los elementos que contienen los datos que necesitas
        libros = soup.find_all('h3', class_='woocommerce-loop-product__title')
        # Itera a través de los libros y extrae los datos
        for libro in libros:
            elem = {}
            titulo = libro.find('a').text.strip()
            url_libro = libro.find('a')['href']

            # Realizo una solicitud a la URL del libro
            try:
                libroResponse = requests.get(url_libro)
                libroResponse.raise_for_status()
                if libroResponse.ok:
                    libroSoup = BeautifulSoup(libroResponse.text, 'html.parser')
                    libroDetalle = libroSoup.find_all('span', class_='woocommerce-Price-amount')
                    # Encuentra el precio en pesos y dólares en el detalle del libro
                    precio_pesos = libroDetalle[1].text.strip()
                    precio_dolar = libroDetalle[1].find_next('span', style="font-size: 1.3em").text.strip()

            except requests.RequestException as libroError:
                print("Ocurrio un error en la URL del libro: ", libroError)
                elemError = {
                    'titulo': titulo,
                    'url_libro': url_libro,
                    'fecha': fecha
                }
                error_top_100_libros.append(elemError)

            elem = {
                'titulo': titulo,
                'url_libro': url_libro,
                'precio_pesos': format_price(precio_pesos),
                'precio_dolar': format_price(precio_dolar),
                'precio_dolar_blue': precio_dolar_blue,
                'fecha': fecha
            }
            # Voy agregando a la lista el libro
            top_100_libros.append(elem)

except requests.RequestException as error:
    print("Ocurrio un error: ", error)

# Tarda aprox 1 min en resolver el bloque
print("La cantidad de libros es: ", len(top_100_libros))
print("la lista de libros obtenidos es: \n", top_100_libros)
print("La lista de errors para los detalles de los libros es: \n", error_top_100_libros)


La cantidad de libros es:  87
la lista de libros obtenidos es: 
 [{'titulo': 'ESTE DOLOR NO ES MIO', 'url_libro': 'https://cuspide.com/producto/este-dolor-no-es-mio/', 'precio_pesos': 9900.0, 'precio_dolar': 27.09, 'precio_dolar_blue': 800, 'fecha': '2023-10-02'}, {'titulo': 'TESIS SOBRE UNA DOMESTICACION', 'url_libro': 'https://cuspide.com/producto/tesis-sobre-una-domesticacion/', 'precio_pesos': 9900.0, 'precio_dolar': 27.09, 'precio_dolar_blue': 800, 'fecha': '2023-10-02'}, {'titulo': 'BEYOND THE STORY ( EDICION EN ESPA/OL )', 'url_libro': 'https://cuspide.com/producto/beyond-the-story-edicion-en-espa-ol/', 'precio_pesos': 7999.0, 'precio_dolar': 21.89, 'precio_dolar_blue': 800, 'fecha': '2023-10-02'}, {'titulo': 'EL VIENTO CONOCE MI NOMBRE', 'url_libro': 'https://cuspide.com/producto/el-viento-conoce-mi-nombre/', 'precio_pesos': 10699.0, 'precio_dolar': 29.27, 'precio_dolar_blue': 800, 'fecha': '2023-10-02'}, {'titulo': 'DESTROZA ESTE DIARIO ( A TODO COLOR )', 'url_libro': 'https:/

In [50]:
# Bloque para conectar y guardar los datos

import mysql.connector

# Conecta a la base de datos MySQL
conn = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="root1234",
    database="cuspide"
)

# Crea una tabla para los datos de los libros
cursor = conn.cursor()
cursor.execute("""
CREATE TABLE IF NOT EXISTS libros (
    id INT AUTO_INCREMENT PRIMARY KEY,
    titulo VARCHAR(255),
    url VARCHAR(255),
    precio_pesos DECIMAL(10, 2),
    precio_dolar DECIMAL(10, 2),
    precio_dolar_blue DECIMAL(10, 2),
    fecha DATE
)
""")

# Inserta los datos de los libros en la tabla
for libro in top_100_libros:
    insert_query = """
    INSERT INTO libros (titulo, url, precio_pesos, precio_dolar, precio_dolar_blue, fecha)
    VALUES (%s, %s, %s, %s, %s, %s)
    """
    values = (libro['titulo'], libro['url_libro'], libro['precio_pesos'], libro['precio_dolar'], libro['precio_dolar_blue'], libro['fecha'])
    cursor.execute(insert_query, values)


# Crea una tabla de auditoría de errores (si no existe)
cursor.execute("""
CREATE TABLE IF NOT EXISTS auditoria_errores (
    id INT AUTO_INCREMENT PRIMARY KEY,
    titulo VARCHAR(255),
    url VARCHAR(255),
    fecha DATE
)
""")

# Inserta los errores en caso que exista
if len(error_top_100_libros) > 0:
    for libro in error_top_100_libros:
        insert_query = """
        INSERT INTO libros (titulo, url, fecha)
        VALUES (%s, %s, %s)
        """
        values = (libro['titulo'], libro['url_libro'], libro['fecha'])
        cursor.execute(insert_query, values)


# Cierra la conexión
conn.close()

In [52]:
# Bloque para corroborar el contenido de la DB

import mysql.connector

# Conecta a la base de datos MySQL
conn = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="root1234",
    database="cuspide"
)

cursor = conn.cursor()
print('------*-------*LIBROS*-------*------*-------*')
# Consultar la base de datos para verificar los datos en la tabla "libros"
cursor.execute('SELECT * FROM libros')
rows = cursor.fetchall()
for row in rows:
    print(row)

print('------*-------*ERRORES*-------*------*-------*')
# Consultar la base de datos para verificar los datos en la tabla "errores"
cursor.execute('SELECT * FROM auditoria_errores')
rows = cursor.fetchall()
for row in rows:
    print(row)

# Cerrar la conexión
conn.close()

------*-------*LIBROS*-------*------*-------*
(1, 'ESTE DOLOR NO ES MIO', 'https://cuspide.com/producto/este-dolor-no-es-mio/', Decimal('9900.00'), Decimal('27.09'), Decimal('800.00'), datetime.date(2023, 10, 2))
(2, 'TESIS SOBRE UNA DOMESTICACION', 'https://cuspide.com/producto/tesis-sobre-una-domesticacion/', Decimal('9900.00'), Decimal('27.09'), Decimal('800.00'), datetime.date(2023, 10, 2))
(3, 'BEYOND THE STORY ( EDICION EN ESPA/OL )', 'https://cuspide.com/producto/beyond-the-story-edicion-en-espa-ol/', Decimal('7999.00'), Decimal('21.89'), Decimal('800.00'), datetime.date(2023, 10, 2))
(4, 'EL VIENTO CONOCE MI NOMBRE', 'https://cuspide.com/producto/el-viento-conoce-mi-nombre/', Decimal('10699.00'), Decimal('29.27'), Decimal('800.00'), datetime.date(2023, 10, 2))
(5, 'DESTROZA ESTE DIARIO ( A TODO COLOR )', 'https://cuspide.com/producto/destroza-este-diario-a-todo-color/', Decimal('11200.00'), Decimal('30.64'), Decimal('800.00'), datetime.date(2023, 10, 2))
(6, 'UNA FAMILIA ANORMA